In [150]:
#importing packages
from collections import Counter, defaultdict
import math
import json
import numpy as np
import os
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from scipy.sparse import csr_matrix
#import urllib.request
#import zipfile


In [151]:
# Read the input
file = "/Users/jponder/Documents/School Notes/Advanced Data Mining/rows.json"
d = d = json.loads(file)

#d = pd.read_json() # the consumer dataset is now a Pandas DataFrame
# Only interested in data with consumer complaints
#d = d[d['Consumer complaint narrative'].notnull()]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
print(d['Consumer complaint narrative'])

In [ ]:
print(d.head())

In [ ]:
def tokenize_string(my_string):
    """ DONE. You should use this in your tokenize function.
    """
    #return re.findall('[\w\-]+', my_string.lower())
    #\W -> Matches any non-alphanumeric character; 
    #this is equivalent to the class [^a-zA-Z0-9_]. 
    
    stemmer = SnowballStemmer("english", ignore_stopwords=True)
    
    no_numbers = my_string.lower().translate({ord(ch): None for ch in '0123456789'})
    word_list = re.sub('\W+', ' ', no_numbers).split()
    stop = set(stopwords.words('english'))
    output = []
    
    for word in word_list:
        if word != "xxxx" and word not in stop:
            word = stemmer.stem(word)
            if len(word) > 2:
                output.append(word)
            
    word_list = output
    
    # preprocessing ->
   
    # remove too low and too high frequency words -> we can't calculate here

    return(word_list)

In [ ]:
def tokenize(data):
    """
    Append a new column to the movies DataFrame with header 'tokens'.
    This will contain a list of strings, one per token, extracted
    from the 'genre' field of each movie. Use the tokenize_string method above.
    Note: you may modify the movies parameter directly; no need to make
    a new copy.
    Params:
      movies...The movies DataFrame
    Returns:
      The movies DataFrame, augmented to include a new column called 'tokens'.
    >>> movies = pd.DataFrame([[123, 'Horror|Romance'], [456, 'Sci-Fi']], columns=['movieId', 'genres'])
    >>> movies = tokenize(movies)
    >>> movies['tokens'].tolist()
    [['horror', 'romance'], ['sci-fi']]
    """
    ###TODO
    
    # step 1 -> do tokenize_string for each row in movies['genres']

    all_words = []
    for row in data['Consumer complaint narrative']:
        #genre_list = re.sub(r'[||)|(]', r' ',row.lower()).split()
        word_list = tokenize_string(row)
        #print(word_list)
        #print(len(genre_list))
        all_words.append(word_list)
    
    # step 2 -> add column tokens in movies
    array = np.array(all_words)
    
    #print(array[:5])
    #print('#list = ',len(array))
    
    new_data = d.assign(tokens = array)
    
    #print(new_movies.head(5))
    return(new_data)  

In [ ]:
def featurize(data):
    """
    Append a new column to the movies DataFrame with header 'features'.
    Each row will contain a csr_matrix of shape (1, num_features). Each
    entry in this matrix will contain the tf-idf value of the term, as
    defined in class:
    tfidf(i, d) := tf(i, d) / max_k tf(k, d) * log10(N/df(i))
    where:
    i is a term
    d is a document (movie)
    tf(i, d) is the frequency of term i in document d
    max_k tf(k, d) is the maximum frequency of any term in document d
    N is the number of documents (movies)
    df(i) is the number of unique documents containing term i
    Params:
      movies...The movies DataFrame
    Returns:
      A tuple containing:
      - The movies DataFrame, which has been modified to include a column named 'features'.
      - The vocab, a dict from term to int. Make sure the vocab is sorted alphabetically as in a2 (e.g., {'aardvark': 0, 'boy': 1, ...})
   
    """
    ###TODO
    #print(movies[:5]) 
    
    #step 1 -> build a vocab and df(term)
    vocab = {}
    vocab_list = []
    df = {}
    
    for tokenization in data['tokens']:
        tokens = list(set(tokenization))
        for term in tokens:
            if term not in vocab.keys():
                vocab.setdefault(term,-1)
             
            if term not in df.keys(): 
                df.setdefault(term,1)
            else :
                df[term] += 1
             
             
    #print('vocab = ', vocab)
    
    vocab_list = sorted(vocab.keys(), key = lambda x:x)
    #print('vocab_list = ', vocab_list)
    
    for i,term in enumerate(vocab_list):
         vocab[term] = i
            
    #        
         
    #print('Sorted vocab = ', sorted(vocab.items()))
    #print('df = ',sorted(df.items(), key=lambda x:x[0]))
    
    # step 2 -> Build a csr_matrix for each row of movies['tokens']
   
    #print('N = ',N)
    
    #[comedy, comedy, comedy, horror]  -> max_k tf(k, d) = 3 
    #[action, comedy,thriller] -> tf(action, d) =1
    # df(i) ->
    #num_features is the total number of unique features across all documents.
    
    N = len(data)
    
    csr_array =[]
    
    for row1 in data['tokens']:
        csr_row = []
        csr_col = []
        csr_data = []
        max_k = 0
       
        max_k = Counter(row1).most_common()[:1][0][1]
        row = list(set(row1))

        #print('removed duplicates =',row)
        for term in row:       
            csr_row.append(0)
            csr_col.append(vocab[term])
            #tfidf(i, d) := tf(i, d) / max_k tf(k, d) * log10(N/df(i))
            tf = Counter(row1)[term]
            #max_k = max_k.most_common()[:1][0][1]
         
            #print('term = %s ---> tf = %d ---> max_k = %d'%(term,tf,max_k))
            tfidf = (tf / max_k) * math.log10(N/df[term])
            csr_data.append(tfidf)
           
         
        #print('csr_row = ',csr_row) 
        #print('csr_col = ',csr_col)
        #print('csr_data=',csr_data)
        X = csr_matrix((csr_data, (csr_row, csr_col)), shape=(1, len(vocab)), dtype=np.float64)
       
        #print('X ->\n',X.toarray())
        csr_array.append(X)
    

    # step 3 -> add column features to movies 
    #print('size of csr_array = ',len(csr_array)) 
    #print('CSR = ',csr_array[:2])  
    new_data = data.assign(features = csr_array)
    #print(new_movies.head(2))
     
    return(new_data,vocab)   
    
    pass

In [ ]:
data = tokenize(d)

In [ ]:
print(data.head(20)['tokens'])
print(data.tail(20)['tokens'])

In [ ]:
data, vocab = featurize(data)

In [ ]:
print('vocab:')
print(sorted(vocab.items())[:500])

In [ ]:
print(len(vocab))